<a href="https://colab.research.google.com/github/Sangeetha2998/Ask-emily/blob/main/emil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (627 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [9]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [10]:
!pip install -U lightrag[ollama]

In [11]:
!pip install streamlit
!pip install hugchat

In [13]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import time
from IPython.display import Markdown, display

# Updated QA template for the Ask Emily project
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User  {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

# Initialize the model client for Ask Emily
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

# Example user query relevant to the Ask Emily system
user_query = "Can you recommend a scenic spot near london?"
output = qa.call(user_query)

# Display the output in Markdown format
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** I'd be happy to recommend a scenic spot near London!

Have you considered visiting Box Hill in Surrey? It's about an hour southwest of London by car or train. The hill offers breathtaking views of the surrounding countryside, and on a clear day, you can even see the iconic London skyline from afar.

If you're looking for something a bit more off-the-beaten-path, I'd suggest exploring the Chiltern Hills Area of Outstanding Natural Beauty (AONB). This picturesque region is dotted with quaint villages, woodlands, and rolling hills. You could stop at one of the many scenic viewpoints or take a leisurely walk along the Chiltern Heritage Trail.

Both of these options are perfect for a day trip from London, and I can even provide you with turn-by-turn GPS navigation if you'd like to drive there yourself!

Would you like me to recommend any specific RV-friendly accommodations in the area, or perhaps help you find an RV technician nearby?

In [ ]:
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

# Define the Ask Emily system prompt for the chatbot
qa_template = r"""<SYS>
You are Ask Emily, an AI-powered Trip Planner, Travel Advisor, GPS, and RV Technician Finder.
You provide personalized trip recommendations, integrate GPS navigation for RVs, and help users find RV repair technicians.
Answer the user's questions based on the context of travel planning and RV maintenance.
</SYS>
:User  {{input_str}}
You:"""

# Define a simple chatbot class for interactive conversation
class ChatBot:
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        self.model_client = model_client
        self.model_kwargs = model_kwargs

    def ask(self, user_input: str) -> str:
        # Format the prompt with the user input
        prompt = qa_template.replace("{{input_str}}", user_input)
        response = self.model_client.call({"input_str": prompt, **self.model_kwargs})
        return response.data if hasattr(response, 'data') else response

# Initialize the model client for Ask Emily
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
chatbot = ChatBot(**model)

# Run the interactive chatbot loop
print("Welcome to Ask Emily! Type 'exit' to end the chat.\n")
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    # Get the response from the chatbot
    output = chatbot.ask(user_query)
    # Display the chatbot response in Markdown
    display(Markdown(f"**Emily:** {output}"))
